## <span style="color:purple">ArcGIS API for Python</span>

<img src="img/Users_04_Analyst.png" style="width: 75%"></img>

## Demo: Analysis Using Drive Time Service Areas

In [1]:
from datetime import datetime
from IPython.display import HTML
import pandas as pd
import arcgis
import getpass

### Authenticate with Web GIS

In [2]:
portals_dict = {
    "esrifederal_gis": r"https://esrifederal.maps.arcgis.com",
    "natgov_gis": r"http://esri-natgov105.eastus.cloudapp.azure.com/arcgis",
    "dot_gis": r"http://dot.esri.com/portal",
    "dev_gis": r"http://govdev.eastus.cloudapp.azure.com/arcgis",
    "local_gis": r"https://anieto.esri.com/arcgis"
}

In [3]:
gis_url = portals_dict["esrifederal_gis"]

In [4]:
if gis_url == portals_dict["esrifederal_gis"]:
    gis_app_id = r"eIIyPrDiF9hsZ6cO"
    print("Attempting to log in to '{0}'...".format(gis_url))
    gis = arcgis.gis.GIS(gis_url, client_id=gis_app_id)
    print("Successfully logged in as: " + gis.properties.user.username)
else:
    gis_username = getpass.getpass(prompt="Username: ")
    gis_pw = getpass.getpass(prompt="Password: ")
    print("Attempting to log in to '{0}'...".format(gis_url))
    gis = arcgis.gis.GIS(gis_url, gis_username, gis_pw, verify_cert=False)
    print("Successfully logged in as: " + gis.properties.user.username)

Attempting to log in to 'https://esrifederal.maps.arcgis.com'...
Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://esrifederal.maps.arcgis.com/sharing/rest/oauth2/authorize?client_id=eIIyPrDiF9hsZ6cO&response_type=code&expiration=-1&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob
Enter code obtained on signing in using SAML: ········
Successfully logged in as: albe9057@esri.com_esrifederal


In [5]:
import arcgis.network as network

In [6]:
service_area_url = gis.properties.helperServices.serviceArea.url
service_area_url

'https://route.arcgis.com/arcgis/rest/services/World/ServiceAreas/NAServer/ServiceArea_World'

In [7]:
sa_layer = network.ServiceAreaLayer(service_area_url, gis=gis)

## Create the facilities layer

In [8]:
fire_station_1_coord = '139.546910,35.695729'
fire_station_2_coord = '139.673726,35.697988'

from arcgis.features import Feature, FeatureSet

f1 = Feature(geometry={'x':float(fire_station_1_coord.split(',')[0]),
                      'y':float(fire_station_1_coord.split(',')[1])})

f2 = Feature(geometry={'x':float(fire_station_2_coord.split(',')[0]),
                      'y':float(fire_station_2_coord.split(',')[1])})

fire_station_fset = FeatureSet([f1,f2], geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

fire_station_fset

{"features": [{"geometry": {"x": 139.54691, "y": 35.695729}}, {"geometry": {"x": 139.673726, "y": 35.697988}}], "spatialReference": {"latestWkid": 4326}, "geometryType": "esriGeometryPoint", "fields": []}

In [9]:
map1 = gis.map('Tokyo', zoomlevel=12)
map1

In [10]:
fire_truck_symbol = {"type":"esriPMS",
                     "url":"http://static.arcgis.com/images/Symbols/SafetyHealth/FireTruck.png",
                     "contentType": "image/png", "width":20, "height":20}

map1.draw(fire_station_fset, symbol=fire_truck_symbol)

## Compute service area

In [11]:
travel_modes = sa_layer.retrieve_travel_modes()
truck_mode = [t for t in travel_modes['supportedTravelModes'] if t['name'] == 'Trucking Time'][0]

result = sa_layer.solve_service_area(fire_station_fset, default_breaks=[5,10,15], 
                                     travel_direction='esriNATravelDirectionFromFacility',
                                     travel_mode=truck_mode)

In [12]:
result.keys()

dict_keys(['messages', 'saPolygons'])

In [13]:
result['saPolygons'].keys()

dict_keys(['fieldAliases', 'geometryType', 'spatialReference', 'features'])

In [14]:
poly_feat_list = []
for polygon_dict in result['saPolygons']['features']:
    f1 = Feature(polygon_dict['geometry'], polygon_dict['attributes'])
    poly_feat_list.append(f1)
poly_feat_list

[{"geometry": {"rings": [[[139.5388164520001, 35.71464347800003], [139.53949165300003, 35.71509361300008], [139.54016685500005, 35.71688652000006], [139.54240989700008, 35.71688652000006], [139.54240989700008, 35.71756172200003], [139.54196357700005, 35.72183036800004], [139.54173851000007, 35.72227668800008], [139.54039192200003, 35.72250175500005], [139.53994178800008, 35.72317695600003], [139.54106330900004, 35.72452354400008], [139.5397167210001, 35.72497367900007], [139.53949165300003, 35.72564506500004], [139.53837013200007, 35.72587013200007], [139.53791999800012, 35.726545334000036], [139.53837013200007, 35.72744178800008], [139.5397167210001, 35.72811698900006], [139.53859520000003, 35.72856712300006], [139.53859520000003, 35.72991371200004], [139.53679847700005, 35.73126030000003], [139.53657341000007, 35.73238563500007], [139.5370197300001, 35.733057022000025], [139.53724479700008, 35.733057022000025], [139.53994178800008, 35.734628677000046], [139.54083824200006, 35.7344036

In [15]:
service_area_fset = FeatureSet(poly_feat_list, 
                         geometry_type=result['saPolygons']['geometryType'],
                         spatial_reference= result['saPolygons']['spatialReference'])

In [16]:
service_area_fset.df

,FacilityID,FromBreak,Name,ObjectID,Shape_Area,Shape_Length,ToBreak,SHAPE
0,1,10,Location 1 : 10 - 15,1,0.002705,0.779069,15,"{'rings': [[[139.5388164520001, 35.71464347800..."
1,1,5,Location 1 : 5 - 10,2,0.001400,0.393997,10,"{'rings': [[[139.55925560000003, 35.7247486110..."
2,1,0,Location 1 : 0 - 5,3,0.000453,0.109327,5,"{'rings': [[[139.55229377700005, 35.7074565890..."
3,2,10,Location 2 : 10 - 15,4,0.003424,1.058613,15,"{'rings': [[[139.705453873, 35.764051437000035..."
4,2,5,Location 2 : 5 - 10,5,0.001749,0.429482,10,"{'rings': [[[139.68995857200002, 35.7177867890..."
5,2,0,Location 2 : 0 - 5,6,0.000554,0.121896,5,"{'rings': [[[139.67423820500005, 35.7097034450..."


## Visualize the service area on the map

In [17]:
colors = {5: [0, 128, 0, 90], 
          10: [255, 255, 0, 90], 
          15: [255, 0, 0, 90]}

fill_symbol = {"type": "esriSFS","style": "esriSFSSolid",
               "color": [115,76,0,255]}

In [18]:
for service_area in service_area_fset.features:
    
    #set color based on drive time
    fill_symbol['color'] = colors[service_area.attributes['ToBreak']]
    
    #set popup
    popup={"title": "Service area", 
            "content": "{} minutes".format(service_area.attributes['ToBreak'])}
    
    map1.draw(service_area.geometry, symbol=fill_symbol, popup=popup)

## Constructing service areas for different times of the day

In [19]:
times = [datetime(2017, 6, 10, h).timestamp() * 1000 
         for h in [6, 10, 14, 18, 22]]

# fire station location
fire_station = '-118.245847, 34.038608'

#loop through each time of the day and compute the service area
sa_results = []
for daytime in times:
    result = sa_layer.solve_service_area(facilities=fire_station, default_breaks=[5,10,15], 
                                         travel_direction='esriNATravelDirectionFromFacility',
                                         time_of_day=daytime, time_of_day_is_utc=False)
    sa_results.append(result)
sa_results

[{'messages': [],
  'saPolygons': {'features': [{'attributes': {'FacilityID': 1,
      'FromBreak': 10,
      'Name': 'Location 1 : 10 - 15',
      'ObjectID': 1,
      'Shape_Area': 0.014464587946275725,
      'Shape_Length': 3.242337385527489,
      'ToBreak': 15},
     'geometry': {'rings': [[[-118.23422431899996, 34.11678886400006],
        [-118.24208450299994, 34.11342048600005],
        [-118.24275779699997, 34.11184883100003],
        [-118.24208450299994, 34.11072731000007],
        [-118.24230956999997, 34.10982704200006],
        [-118.24253463699995, 34.10982704200006],
        [-118.24298286399994, 34.10937690700007],
        [-118.24365615799996, 34.10960197400004],
        [-118.24455451999995, 34.11117363000005],
        [-118.24522972099999, 34.11139869700003],
        [-118.24837303199996, 34.10915565500005],
        [-118.24859809899999, 34.10354042100005],
        [-118.24724960299994, 34.10151863100003],
        [-118.24500465399996, 34.10129356400006],
        [-1

In [20]:
LA_fset_list=[]
for result in sa_results:
    poly_feat_list = []
    for polygon_dict in result['saPolygons']['features']:
        f1 = Feature(polygon_dict['geometry'], polygon_dict['attributes'])
        poly_feat_list.append(f1)
        
    service_area_fset = FeatureSet(poly_feat_list, 
                         geometry_type=result['saPolygons']['geometryType'],
                         spatial_reference= result['saPolygons']['spatialReference'])
    
    LA_fset_list.append(service_area_fset)
LA_fset_list    

[{"features": [{"geometry": {"rings": [[[-118.23422431899996, 34.11678886400006], [-118.24208450299994, 34.11342048600005], [-118.24275779699997, 34.11184883100003], [-118.24208450299994, 34.11072731000007], [-118.24230956999997, 34.10982704200006], [-118.24253463699995, 34.10982704200006], [-118.24298286399994, 34.10937690700007], [-118.24365615799996, 34.10960197400004], [-118.24455451999995, 34.11117363000005], [-118.24522972099999, 34.11139869700003], [-118.24837303199996, 34.10915565500005], [-118.24859809899999, 34.10354042100005], [-118.24724960299994, 34.10151863100003], [-118.24500465399996, 34.10129356400006], [-118.24455451999995, 34.10017204300004], [-118.24320793199996, 34.10017204300004], [-118.24208450299994, 34.09882164000004], [-118.24208450299994, 34.09860038800008], [-118.24230956999997, 34.097925186000055], [-118.24185943599997, 34.097024918000045], [-118.24028778099995, 34.09680366500004], [-118.23916625999999, 34.095232010000075], [-118.23624610899998, 34.09365654

In [21]:
map2= gis.map("Los Angeles, CA")
map2

In [26]:
import time
map2.clear_graphics()

times = ['6 am', '10 am', '2 pm', '6 pm', '10 pm']
j=0
time.sleep(2)

for fset in LA_fset_list:
    print(times[j])
    map2.draw(fset)
    j+=1
    time.sleep(2)

6 am
10 am
2 pm
6 pm
10 pm
